In [114]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import numpy as np

In [115]:
s = "This concept of distance is not restricted to two dimensions."
# list_s = ["This concept of distance is not restricted to two dimensions.", "This concept of distance is not restricted to two dimensions."]
corpus = [
        'This is the first document.',
        'This is the second second document.',
        'And the third one.',
        'Is this the first document?',
        'It is not difficult to imagine the figure above translated into three dimensions.',
        'We can persuade ourselves that the measure of distance extends to an arbitrary number of dimensions;',
        ]

In [116]:
stemmer = PorterStemmer()

def stem(text):
    text_stem = [stemmer.stem(token) for token in text.split(' ')]
    text_stem_join = ' '.join(text_stem)
    return text_stem_join
corpus_stem = list(map(stem, corpus))

## TODO - Show the tfidf.get_stop_words()

In [117]:
tfidf = TfidfVectorizer(norm='l2', min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, stop_words='english')

X = tfidf.fit_transform([s] + corpus_stem)
print(tfidf.get_feature_names())
print(X.toarray())

['abov', 'arbitrari', 'concept', 'difficult', 'dimensions', 'distanc', 'distance', 'document', 'extend', 'figur', 'imagin', 'measur', 'number', 'ourselv', 'persuad', 'restricted', 'second', 'thi', 'translat']
[[ 0.          0.          0.54286774  0.          0.34041718  0.
   0.54286774  0.          0.          0.          0.          0.          0.
   0.          0.          0.54286774  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.70710678  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.70710678  0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.32808134  0.          0.          0.          0.          0.          0.
   0.          0.          0.8858472   0.32808134  0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.   

In [118]:
print('cosine distance 0 -> 1: ', cosine(X[0].toarray(), X[1].toarray()))
print('cosine distance 0 -> 2: ', cosine(X[0].toarray(), X[2].toarray()))
print('cosine similarity 0 -> 1: ', cosine_similarity(X[0].toarray(), X[1].toarray()))
print('cosine similarity 0 -> 2: ', cosine_similarity(X[0].toarray(), X[2].toarray()))
print('cosine similarity 0 -> 3: ', cosine_similarity(X[0].toarray(), X[3].toarray()))
print('cosine similarity 0 -> 4: ', cosine_similarity(X[0].toarray(), X[4].toarray()))
print('cosine similarity 0 -> 5: ', cosine_similarity(X[0].toarray(), X[5].toarray()))
print('cosine similarity 0 -> 6: ', cosine_similarity(X[0].toarray(), X[6].toarray()))
# print(X)

cosine distance 0 -> 1:  1.0
cosine distance 0 -> 2:  1.0
cosine similarity 0 -> 1:  [[ 0.]]
cosine similarity 0 -> 2:  [[ 0.]]
cosine similarity 0 -> 3:  [[ 0.]]
cosine similarity 0 -> 4:  [[ 0.]]
cosine similarity 0 -> 5:  [[ 0.09191891]]
cosine similarity 0 -> 6:  [[ 0.07850767]]


# TODO - Create a sparse matrix of similarities between every article

# TODO - Plot the sparse matrix of similarities using a heatmap

# TODO - Create a wordcloud of the feature_names